In [42]:
# here, we combine all .csv files into the single file total_state_data.csv
# all .csv files to be combined should be placed in ./Data directory

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df1 = pd.read_csv('./Data/1990-2020-final.csv')
df2 = pd.read_csv('./Data/monthly-emissions-1996-2024-final.csv')
df3 = pd.read_csv('./Data/monthly-weather-1990-2019-final.csv')
df4 = pd.read_csv('./Data/gdp-1997-2023-final-final.csv')

In [43]:
df4.head(50)

,Unnamed: 0,state,date,gdp_rel_2017
0,0,Alabama,1997-01-01,154700.0
1,1,Alabama,1997-02-01,154700.0
2,2,Alabama,1997-03-01,154700.0
3,3,Alabama,1997-04-01,154700.0
4,4,Alabama,1997-05-01,154700.0
5,5,Alabama,1997-06-01,154700.0
6,6,Alabama,1997-07-01,154700.0
7,7,Alabama,1997-08-01,154700.0
8,8,Alabama,1997-09-01,154700.0
9,9,Alabama,1997-10-01,154700.0


In [44]:
# remove duplicate indexing columns

df1 = df1.drop('Unnamed: 0', axis=1)
df2 = df2.drop('Unnamed: 0', axis=1)

In [45]:
total_df = df1.merge(df2, on=['state','date'], how='inner').merge(df3, on=['state','date'], how='inner')

In [46]:
total_df.to_csv('./Data/total_state_data.csv')